In [541]:
import numpy as np
from scipy.optimize import fsolve
import time
import copy
import matplotlib.pyplot as plt
import seaborn as sns

In [542]:
def plot_accu_change(Y_list, target):
    # 设置Seaborn主题
    sns.set_theme(style="whitegrid", palette="muted")

    # 创建画布
    # plt.figure(figsize=(10, 6))

    # 绘制折线图
    sns.lineplot(Y_list, marker='o', linewidth=2, label="Epochs vs. Value")

    # 添加横线
    plt.axhline(y=target, color='r', linestyle='--', label=f"Target Value: {target}")   

    # 设置轴标签和标题
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Value', fontsize=14)
    # plt.title('', fontsize=16)
    # 美化图表
    plt.xticks(np.arange(0, len(Y_list)),fontsize=12)
    plt.yticks(fontsize=12)
    plt.grid(True)

    # 添加图例
    plt.legend()

    # 显示图形
    plt.tight_layout()

In [543]:
def get_initial_guess(m:np.array):
    J, L, _ = (np.shape(m))
    X = np.ones((J, L))
    
    Y = np.ones((J, 1))
    
    for j in range(J):
        Y[j] = np.sum(m[j])

    return X, Y, J, L
    

In [544]:
# 定义方程组
def equations(guess:np.array,I:np.array, P, J, L):
    X = np.reshape(guess[:J*L], (J, L))
    Y = np.reshape(guess[J*L:], (J, 1))
    eqs = list()
    
    for j in range(J):
        pred_time = 0
        for l in range(L):
            comp = I[j][l][0]
            comm = I[j][l][1]
            pred_time += (comp + comm * X[j][l])        
        eqs.append(pred_time - Y[j][0])
    
    
    for j in range(J):
        for l in range(L):
            scale = 1
            for other_flow in range(J):
                comm = I[other_flow][l][1]
                if P[j][l] < P[other_flow][l]:
                    single_scale = Y[other_flow][0] / (Y[other_flow][0] - comm * X[other_flow][l])
                    
                else:
                    single_scale = 1
                scale *= single_scale
                # if scale > 1:
                #     break
            eqs.append(scale - X[j][l])
    # print(eqs)
    return eqs

In [545]:
def update_comm_time(m:np.array, P, J, L):
    X, Y, J, L = get_initial_guess(m)
    guess = np.concatenate((X.flatten(), Y.flatten()))
    result = fsolve(equations, guess, args=(m, P, J, L))
    X = np.reshape(result[:J*L], (J, L))
    Y = np.reshape(result[J*L:], (J, 1))
    return X, Y

In [546]:
def generate_random_I(J, L, min_v, max_v):
    # 生成范围在[min_v, max_v]内的随机浮动值
    arr = np.random.uniform(min_v, max_v, size=(J, L, 2))

    # 确保每行不全是[0, 0]
    for i in range(J):
        while np.all(arr[i] == [0, 0]):
            arr[i] = np.random.uniform(min_v, max_v, size=(L, 2))

    # 使得每列至少有一半的[0, 0]
    for j in range(L):
        # 计算每列中[0, 0]的数量
        zero_count = np.sum(np.all(arr[:, j] == [0, 0], axis=1))

        # 如果[0, 0]数量小于一半，则添加[0, 0]元素
        while zero_count < J // 2:
            # 选择一个非[0, 0]的元素进行替换
            row_to_replace = np.random.choice(np.where(np.all(arr[:, j] != [0, 0], axis=1))[0])
            arr[row_to_replace, j] = [0, 0]
            zero_count = np.sum(np.all(arr[:, j] == [0, 0], axis=1))

    return arr

In [547]:
def solve(I, P):
    initial_X, initial_Y, J, L = get_initial_guess(I)
    initial_guess = np.concatenate([initial_X.flatten(),initial_Y.flatten()])

    solution = fsolve(equations, initial_guess, args=(I,P,J,L))
    solution_X = np.reshape(solution[:J*L], (J, L))
    solution_Y = np.reshape(solution[J*L:], (J, 1))

    for j in range(J):
        for l in range(L):
            if list(I[j][l]) == [0,0]:
                solution_X[j][l] = 0
    
    # print("X的解为\n", solution_X)
    
    # print("初始Y为\n", initial_Y.reshape(1,J)[0].tolist())
    # print("竞争后Y为\n", solution_Y.reshape(1,J)[0].tolist())
    # print("scale为\n", (solution_Y/initial_Y).reshape(1,J)[0].tolist())
    # print("scale之和为", np.sum(solution_Y/initial_Y))
    res = np.sum(solution_Y/initial_Y) / J
    # print("precise:",res)
    return res
    

In [548]:
def solve_approximate(I, P, epochs):
    J, L, _ = np.shape(I)
    Y = np.ones((J, 1))
    for j in range(J):
        Y[j][0] = np.sum(I[j])
    print("initial_Y",Y.T[0].tolist())
    print("initial_I",I)
    
    initial_I = copy.deepcopy(I)
    initial_Y = copy.deepcopy(Y)
    
    link_scale_arr = np.ones((J, L))
    #indices = np.where(P[:, 0] == 1)[0]
    # epochs = 1
    for epoch in range(epochs):
        # use the I from the last epoch
        newest_I = copy.deepcopy(I)
        
        I = copy.deepcopy(initial_I)

        for l in range(L):
            curr_prio = J - 1
            reduce_factor = 1
            save_factor = 1
            while curr_prio >= 0:
                indices = np.where(P[:, l] == curr_prio)[0][0]
                
                comm = I[indices][l][1]
                if comm == 0:
                    curr_prio -= 1
                    continue
                new_comm = comm * reduce_factor
                
                I[indices,l,1] = new_comm
                
                # record_advance[indices,l,1] *= save_factor

                if Y[indices,0] - newest_I[indices][l][1] == 0:
                    curr_prio -= 1
                    continue
                # save_factor = reduce_factor
                reduce_factor *= (Y[indices,0] / (Y[indices,0] - newest_I[indices][l][1]))
                    
                curr_prio -= 1
                
        for j in range(J):
            Y[j][0] = np.sum(I[j])
        
        # print("record_advance",record_advance)
        print("I",I)
        # print("Delta I",I - initial_I)
        # print("delta Y",(Y-initial_Y).T[0].tolist())
                
    res = np.sum(Y/initial_Y)
    # print("initial",initial_Y.T[0].tolist())
    print("approxi:",Y.T[0].tolist())
    print("scale:", ((Y-initial_Y)/initial_Y).T[0].tolist())
    return res
    # return I, Y, initial_Y
    

In [549]:
def solve_approximate_double(I, P, epochs):
    J, L, _ = np.shape(I)
    Y = np.ones((J, 1))
    Y_list = []
    for j in range(J):
        Y[j][0] = np.sum(I[j])
    #print("initial_Y",Y.T[0].tolist())
    #print("initial_I",I)
    
    initial_I = copy.deepcopy(I)
    initial_Y = copy.deepcopy(Y)
    
    # add one row the record the lowest prio of job in each link,
    # for when we decrease a job flow of prio 1, it will be influenced by the job flow of prio 0
    link_scale_arr = np.ones((J+1, L))
    increase_prio_arr = np.ones((J, L))
    decrease_prio_arr = np.ones((J, L))
    reward_matrix_list = []
    # add all the epoch's reward_matrix
    final_reward_matrix = np.zeros((J, L))
    for epoch in range(epochs):
        # use the I from the last epoch
        newest_I = copy.deepcopy(I)
        # pre-compute the scale factor of each link
        for l in range(L):
            curr_prio = J
            while curr_prio >= 0:
                if curr_prio == J:
                    link_scale_arr[curr_prio][l] = 1
                else:
                    indices = np.where(P[:, l] == curr_prio)[0][0]
                    #print("indices",indices)
                    if Y[indices,0] == newest_I[indices][l][1]:
                        scale = 1
                    else:
                        scale = (Y[indices,0] / (Y[indices,0] - newest_I[indices][l][1]))
                    link_scale_arr[curr_prio][l] = link_scale_arr[curr_prio+1][l] * scale

                curr_prio -= 1
        # print("link_scale_arr",link_scale_arr)
        
        I = copy.deepcopy(initial_I)
        # First, we calculate the newest I, attention this step must be done after the above two steps
        # for it will change the value of I, which will be used in the former two steps, demanding the 
        # origin values
        for l in range(L):
            for j in range(J):
                prio = P[j][l]
                comm = I[j][l][1]
                if comm == 0:
                    continue
                I[j][l][1] = comm * link_scale_arr[prio+1][l]
        
           
        for j in range(J):
            Y[j][0] = np.sum(I[j])
        Y_list.append(np.sum(Y/initial_Y) / J)
        # print("===epoch %d===" % epoch)
        # print("I")
        # print(I)
        # print("Y")
        # print(Y)
        
        # print("dont change", I[:,:,1])
        # print("increase_prio_arr",increase_prio_arr)
        # print("decrease_prio_arr",decrease_prio_arr)
        
        # here for point the direction of the next prio matrix, we use
        # the increase_prio_arr and decrease_prio_arr to decide the direction
        # one increase 1 prio, another which upper rightly 1 prio decrease 1 prio
        # see the diff of the two value of the two matrix, if the diff is positive
        # then we should increase the prio, otherwise decrease the prio
        
        # reward_matrix[j][l] means if we increase the prio of job j in link l, how much reward we can get
        # it consists of two parts, one is the increase of the job j, another is the decrease of the peer job
        
        # increase the prio, we only pay attention to the diff
        for l in range(L):
            for j in range(J):
                prio = P[j][l]
                if prio == J - 1:
                    increase_prio_arr[j][l] = 0
                else:
                    increase_prio_arr[j][l] = I[j][l][1] / (link_scale_arr[prio+1][l] / link_scale_arr[prio+2][l]) - I[j][l][1]
        increase_prio_arr = increase_prio_arr / initial_Y
        # decrease the prio, we only pay attention to the diff
        for l in range(L):
            for j in range(J):
                prio = P[j][l]
                if prio == 0:
                    decrease_prio_arr[j][l] = 0
                else:
                    decrease_prio_arr[j][l] =  I[j][l][1] * (link_scale_arr[prio-1][l] / link_scale_arr[prio][l]) - I[j][l][1]
        decrease_prio_arr = decrease_prio_arr / initial_Y
            
        
        reward_matrix = np.zeros((J, L))
        for l in range(L):
            for j in range(J):
                prio = P[j][l]
                if prio == J - 1:
                    reward_matrix[j][l] = 0
                    continue
                peer_prio_indice = np.where(P[:, l] == prio + 1)[0][0]
                increase_reward  = increase_prio_arr[j][l]
                decrease_penalty = decrease_prio_arr[peer_prio_indice][l]
                reward_matrix[j][l] = increase_reward + decrease_penalty
        # print("reward_matrix")
        # print(reward_matrix)
        reward_matrix_list.append(reward_matrix)
    
    for reward_matrix in reward_matrix_list[0:1]:
        final_reward_matrix += reward_matrix
    # print("final_reward_matrix",final_reward_matrix)
    job_index_inc, link_index = np.unravel_index(final_reward_matrix.argmin(), final_reward_matrix.shape)
    prio_to_inc = P[job_index_inc][link_index]
    # print("prio_to_inc",prio_to_inc)
    if final_reward_matrix[job_index_inc][link_index] > 0 or prio_to_inc == J - 1:
        print("No more to increase")
        job_index_dec = None
    else:
        job_index_dec = np.where(P[:, link_index] == prio_to_inc + 1)[0][0]

    res = np.sum(Y/initial_Y) / J

    print("scale:", ((Y)/initial_Y).T[0].tolist())
    
    
    # # we use the second method tio decide the direction of the next prio matrix
    # # 1. the increase ratio of the current job flow \
    # # 2. the decrease ratio of the peer job flow 
    # # 3. the affect of the two jobs' change
    # for l in range(L):
    #     curr_prio = J
    #     while curr_prio >= 0:
    #         if curr_prio == J:
    #             link_scale_arr[curr_prio][l] = 1
    #         else:
    #             indices = np.where(P[:, l] == curr_prio)[0][0]
    #             #print("indices",indices)
    #             if Y[indices,0] == initial_I[indices][l][1]:
    #                 scale = 1
    #             else:
    #                 scale = (Y[indices,0] / (Y[indices,0] - initial_I[indices][l][1]))
    #             link_scale_arr[curr_prio][l] = link_scale_arr[curr_prio+1][l] * scale

    #         curr_prio -= 1
    # print("newest_I")
    # print(newest_I)
    # print("Y")
    # print(Y)
    # print("link_scale_arr")
    # print(link_scale_arr)
    
    
    
    
    return res, job_index_inc, job_index_dec, link_index, Y_list
    

In [550]:
def main(I, P, opti_steps, approx_epochs):
    
    approx_res_list = []
    for i in range(opti_steps):
        start_time = time.time()
        print("*"*10,"step ",i,"*"*10)
        approx_res, job_index_inc, job_index_dec, link_index, Y_list = solve_approximate_double(I, P, approx_epochs)
        
        # print("approx耗时%.8fms"%((time.time() - start_time)*1000))
        print("approx:",approx_res)
        print("in link ", link_index, "increase ",job_index_inc, "decrease ",job_index_dec)
        approx_res_list.append(approx_res)
        if job_index_dec is None:
            break
        else:
            P[job_index_inc][link_index] += 1
            P[job_index_dec][link_index] -= 1
        print(P)

    print("approx_res_list", [x.tolist() for x in approx_res_list])
    sns.lineplot(x=range(len(approx_res_list)), y=approx_res_list, marker='o', linewidth=2)

    # 设置轴标签和标题
    plt.xlabel('Step', fontsize=14)
    plt.ylabel('Average Reduce', fontsize=14)
    plt.title('Average Reduce vs. Step', fontsize=16)

    # 美化线图
    plt.grid(True)  # 打开网格线
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)

    # 显示图形
    plt.tight_layout()

In [ ]:
I_test = np.array(
    [
        [[1,3],[0,0]],
        [[1,8],[0,1]],
        [[0,0],[1,2]],
    ],dtype=np.float64
)

P_test = np.array(
    [
        [1,0],
        [2,1],
        [0,2]
    ]
)
print("*"*20)
start_time = time.time()

precise_res = np.ones((I_test.shape[0],1))
precise_res = solve(I_test, P_test)
print("precise耗时%.8fms"%((time.time() - start_time)*1000))

start_time = time.time()
approx_res, job_index_inc, job_index_dec, link_index, Y_list = solve_approximate_double(I_test, P_test, 4)
print("approx耗时%.8fms"%((time.time() - start_time)*1000))

print("precise:",np.mean(precise_res))
print("approx:",np.mean(approx_res))
print("error",np.mean(precise_res-approx_res)/np.mean(precise_res))

print(Y_list)
plot_accu_change(Y_list, precise_res)
# I = copy.deepcopy(I_test)
# P = copy.deepcopy(P_test)
# main(I,P,10,2)

In [ ]:
import numpy as np

# 设置随机种子（可选）
# np.random.seed(41)

# 定义 J 和 P
J = 10  # 你可以根据需求调整
L = 20  # 你可以根据需求调整

debug = 0
# 生成 J*P 的随机数组，元素为 1*2，且每个元素在 0-5 之间
if debug:
    random_I = np.array([
        [[1, 3], [0, 1], [0, 0]],
        [[0, 0], [1, 4], [3, 1]],
        [[0, 1], [1, 1], [0, 1]],
        [[2, 2], [7, 8], [0, 0]],
        [[4, 3], [4, 2], [1, 6]], 
    ],dtype=np.float64)
    random_P = np.array([
        [4, 4, 0],
        [3, 3, 1],
        [2, 2, 2],
        [1, 1, 3],
        [0, 0, 4],
    ])
else:
    random_I = generate_random_I(J, L, 0, 5)
    random_P = np.array([np.random.permutation(J) for _ in range(L)]).T
    # random_P = np.tile(np.arange(J), (L, 1)).T
    # print(random_P)
    # print(random_I)

print("*"*20)
start_time = time.time()

precise_res = np.ones((J,1))
precise_res = solve(random_I, random_P)
print("precise耗时%.8fms"%((time.time() - start_time)*1000))

start_time = time.time()
approx_res, job_index_inc, job_index_dec, link_index, Y_list = solve_approximate_double(random_I, random_P, 10)
print("approx耗时%.8fms"%((time.time() - start_time)*1000))

plot_accu_change(Y_list, precise_res)
print("*"*20)
print("precise:",np.mean(precise_res))
print("approx:",np.mean(approx_res))
print("error",np.mean(precise_res-approx_res)/np.mean(precise_res))
print("in link ", link_index, "increase ",job_index_inc, "decrease ",job_index_dec)


In [ ]:
I = copy.deepcopy(random_I)
P = copy.deepcopy(random_P)
main(I, P, 10, 10)

In [44]:
import numpy as np
import random

def get_min_and_random(arr, n):
    # 获取所有非零元素的索引和对应的值
    non_zero_elements = [(i, j, arr[i][j]) for i in range(arr.shape[0]) for j in range(arr.shape[1]) if arr[i][j] != 0]
    
    # 如果非零元素的数量少于n，调整n的值
    n = min(n, len(non_zero_elements))

    # 获取最小的n个非零元素
    non_zero_elements.sort(key=lambda x: x[2])  # 按值排序
    min_n_elements = non_zero_elements[:n]
    min_n_indices = [(i, j) for i, j, value in min_n_elements]

    # 创建新数组，并将最小的n个元素位置置为0
    modified_arr = arr.copy()
    for i, j in min_n_indices:
        modified_arr[i][j] = 0

    # 获取剩余非零元素
    remaining_non_zero_elements = [(i, j, modified_arr[i][j]) for i in range(modified_arr.shape[0]) for j in range(modified_arr.shape[1]) if modified_arr[i][j] != 0]

    # 随机选择剩余非零元素中的n//2个
    remaining_n = min(n // 2 + 1, len(remaining_non_zero_elements))
    random_selected_elements = random.sample(remaining_non_zero_elements, remaining_n)
    random_selected_indices = [(i, j) for i, j, value in random_selected_elements]

    # 返回所有选中的元素值和对应的索引
    selected_values = [value for i, j, value in min_n_elements] + [value for i, j, value in random_selected_elements]
    selected_indices = min_n_indices + random_selected_indices

    return selected_values, selected_indices

# 示例
arr = np.array([[0, -1, 0], 
                [0, 0, 0], 
                [0, 0, 0]])
n = 1
selected_values, selected_indices = get_min_and_random(arr, n)

print("Selected values:", selected_values)
print("Selected indices:", selected_indices)


Selected values: [np.int64(-1)]
Selected indices: [(0, 1)]
